In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# modify to where you store your project data including utils
datadir = "/content/drive/My Drive/cs445/445 final/" 

In [ ]:
import numpy as np
import cv2
from scipy.spatial import Delaunay
from imutils import face_utils
import imutils
import dlib

# function that utilizes dlib to create landmarks
def create_landmarks(im_path):
    # Initialize face detector and shape predictor
    detector_value = dlib.get_frontal_face_detector()
    predictor_value = dlib.shape_predictor("/Users/rohansathe/Downloads/shape_predictor_68_face_landmarks.dat")

    # Read in the image
    im = cv2.imread(im_path)

    # Detect faces in the image
    rectangles = detector_value(im, 1)

    # Get face landmarks
    im_landmark_arr = []
    for (i, rect) in enumerate(rectangles):
        shape = predictor_value(im, rect)
        # convert shape predictor into something readable
        shape = face_utils.shape_to_np(shape)
        for (x, y) in shape:
            im_landmark_arr.append((x,y))
    
    im_landmark_arr = np.array(im_landmark_arr)
    return im_landmark_arr
    
# this function will actually warp the two images together using the original image landmarks and the weighted image landmarks
def warp_image(im, landmark_list1, weighted_landmark_list, dst_triangulation):
    im_out = im.copy()

    # Warp each triangle from first im landmarks to second im landmarks 
    for i in range(len(dst_triangulation)):
        # finds points that correspond to indices of delaunay triangles
        landmark_list1_app = landmark_list1[dst_triangulation[i]]
        weighted_landmark_list_app = weighted_landmark_list[dst_triangulation[i]]

        # Put bonding rectangles around the first and second triangles
        rect1 = cv2.boundingRect(np.float32([landmark_list1_app]))
        rect_weighted = cv2.boundingRect(np.float32([weighted_landmark_list_app]))

        # Initialize new lists
        bounded_lmlist1 = []
        bounded_lmlistweighted = []

        # Get new triangle coordinates in the bounding box
        for i in range(3):
            bounded_lmlist1.append((landmark_list1_app[i][0] - rect1[0], landmark_list1_app[i][1] - rect1[1]))
            bounded_lmlistweighted.append((weighted_landmark_list_app[i][0] - rect_weighted[0], weighted_landmark_list_app[i][1] - rect_weighted[1]))

         # Create and Fill mask for second image (i.e. bounded_lmlistweighted)
        mask = np.zeros((rect_weighted[3], rect_weighted[2], 3), dtype=np.float32)
        cv2.fillConvexPoly(mask, np.int32(bounded_lmlistweighted), (1.0, 1.0, 1.0), 16, 0)

        # apply bounding box constraints to first image
        im_cropped = im[rect1[1]:rect1[1] + rect1[3], rect1[0]:rect1[0] + rect1[2]]

        # affine transformation using bounded landmarks
        im_size = (rect_weighted[2], rect_weighted[3])
        M = cv2.getAffineTransform(np.float32(bounded_lmlist1), np.float32(bounded_lmlistweighted))
        # warp affine transform
        warped_im = cv2.warpAffine(im_cropped, M, im_size, borderMode=cv2.BORDER_REFLECT_101)

        # Blend the warped image patch with the output image
        im_out[rect_weighted[1]:rect_weighted[1]+rect_weighted[3], rect_weighted[0]:rect_weighted[0]+rect_weighted[2]] = im_out[rect_weighted[1]:rect_weighted[1]+rect_weighted[3], rect_weighted[0]:rect_weighted[0]+rect_weighted[2]] * (1 - mask) + warped_im* mask

    return im_out

# primary driver function
def fin_func():
    # Define file paths for input images and put them into enumerated list
    filename_list = [
        '/Users/rohansathe/Documents/CS445/MP5/nbajpeg/beal.jpeg',
        '/Users/rohansathe/Documents/CS445/MP5/nbajpeg/brunson.jpeg',
        '/Users/rohansathe/Documents/CS445/MP5/nbajpeg/devinbooker.jpeg',
        '/Users/rohansathe/Documents/CS445/MP5/nbajpeg/embiid.jpeg',
        '/Users/rohansathe/Documents/CS445/MP5/nbajpeg/jimmybuckets.jpeg',
        '/Users/rohansathe/Documents/CS445/MP5/nbajpeg/jokic.jpeg',
        '/Users/rohansathe/Documents/CS445/MP5/nbajpeg/kevindurant.jpg',
        '/Users/rohansathe/Documents/CS445/MP5/nbajpeg/klay.jpeg',
        '/Users/rohansathe/Documents/CS445/MP5/nbajpeg/lebron.jpeg',
        '/Users/rohansathe/Documents/CS445/MP5/nbajpeg/tatum.jpeg'
    ]

    num = 1
    # Iterate through each pair of consecutive images
    for i in range(len(filename_list)-1):
        print(i)
        # Read the images
        im1 = cv2.imread(filename_list[i])
        im2 = cv2.imread(filename_list[i+1])

        # Get face landmarks
        landmark_list1 = create_landmarks(filename_list[i])
        landmark_list2 = create_landmarks(filename_list[i+1])
        landmark_list1 = np.array(landmark_list1)
        landmark_list2 = np.array(landmark_list2)

        # Compute average landmarks
        average_landmarks = (landmark_list1 + landmark_list2) / 2 

        # Delaunay triangulation with average landmarks between original and second images 
        triangulation = Delaunay(average_landmarks).simplices
        alpha = 0.0
        while alpha <= 1.0:
            # applies a certain level of alpha to original landmark list v. final --> the smaller alpha is the less of that image is included in the morph 
            weighted_landmark_list = (1.0 - alpha) * landmark_list1 + alpha * landmark_list2
            warped_image1 = warp_image(im1, landmark_list1, weighted_landmark_list, triangulation)
            warped_image2 = warp_image(im2, landmark_list2, weighted_landmark_list, triangulation)

            # similar process to one of our earlier MPs
            blended_image = (1.0 - alpha) * warped_image1 + alpha * warped_image2
            cv2.imwrite("/Users/rohansathe/Documents/CS445/FinalProject/nbaimagefolder/imfin" + f'{num:05}' + ".jpg", blended_image)
            alpha = alpha + 0.01
            num = num + 1
    return 0

if __name__ == '__main__':
    blended_fin = fin_func()

In [ ]:
COMMAND USED TO GENERATE VIDEO FROM MORPHED IMAGES --> YOU CAN TOGGLE WITH THE ALPHA INCREASE TO GENERATE MORE OR LESS WARPING

#ffmpeg -framerate 20 -i '/Users/rohansathe/Documents/CS445/FinalProject/nbaimagefolder/imfin%05d.jpg' -c:v libx264 -vf fps=20 -pix_fmt yuv420p '/Users/rohansathe/Documents/CS445/FinalProject/finalnbavideo.mp4'